# DEPENDÊNCIAS

In [ ]:
! pip install spacy
! pip install nltk
! pip install pandas
! pip install pdfplumber
! pip install chromadb
! pip install sentence-transformers
! pip install openpyxl
! pip install python-docx
! python -m spacy download pt_core_news_sm
! pip install langchain-text-splitters

# Carrega PDF

In [4]:
import pdfplumber

pdf_path = "./documents/sample_vangogh.pdf"

def load_pdf(filepath):
    with pdfplumber.open(filepath) as pdf:
        text = ""
        for page in pdf.pages:
            text += page.extract_text() or ""
    return text

raw_text = load_pdf(pdf_path)

print("Prévia do texto lido:\n")
print(raw_text[:500])


Prévia do texto lido:

Prólogo
Um coração fanático
Theo imaginou o pior. A mensagem dizia apenas que Vincent tinha “se ferido”.
Enquanto corria até a estação para pegar o próximo trem até Auvers, seus pensa-
mentos vagavam de trás para diante. Da última vez que havia recebido uma mensa-
gem parecida, era um telegrama de Paul Gauguin informando que Vincent estava
“gravemente enfermo”. Theo chegara à cidade meridional de Arles e encontrara o
irmão na enfermaria de um hospital, com a cabeça enfaixada e perdido em desvari


# PLN

## Lowercasing

In [5]:
text_lower = raw_text.lower()

print("Texto em lowercase (prévia):\n")
print(text_lower[:800])


Texto em lowercase (prévia):

prólogo
um coração fanático
theo imaginou o pior. a mensagem dizia apenas que vincent tinha “se ferido”.
enquanto corria até a estação para pegar o próximo trem até auvers, seus pensa-
mentos vagavam de trás para diante. da última vez que havia recebido uma mensa-
gem parecida, era um telegrama de paul gauguin informando que vincent estava
“gravemente enfermo”. theo chegara à cidade meridional de arles e encontrara o
irmão na enfermaria de um hospital, com a cabeça enfaixada e perdido em desvarios.
dessa vez, o que encontraria ao final da viagem?
em momentos assim — e eram muitos —, theo retornava às lembranças do
vincent que tinha conhecido no passado: um irmão mais velho ardoroso e irrequie-
to, mas também cheio de brincadeiras animadas, uma enorme afinidade e uma infi-
nita capacidade d


## Remover caracteres especiais

In [6]:
import re
import unicodedata

def remove_special_chars(text):
    # Normaliza acentos (NFKD)
    text = unicodedata.normalize("NFKD", text)
    text = text.encode("ASCII", "ignore").decode("utf-8")

    # Remove caracteres que não sejam letras ou espaço
    text = re.sub(r"[^a-z0-9.,\s]", " ", text)

    # Remove múltiplos espaços
    text = re.sub(r"\s+", " ", text)

    return text.strip()

text_clean = remove_special_chars(text_lower)

print("Texto sem caracteres especiais (prévia):\n")
print(text_clean[:800])


Texto sem caracteres especiais (prévia):

prologo um coracao fanatico theo imaginou o pior. a mensagem dizia apenas que vincent tinha se ferido. enquanto corria ate a estacao para pegar o proximo trem ate auvers, seus pensa mentos vagavam de tras para diante. da ultima vez que havia recebido uma mensa gem parecida, era um telegrama de paul gauguin informando que vincent estava gravemente enfermo. theo chegara a cidade meridional de arles e encontrara o irmao na enfermaria de um hospital, com a cabeca enfaixada e perdido em desvarios. dessa vez, o que encontraria ao final da viagem em momentos assim e eram muitos , theo retornava as lembrancas do vincent que tinha conhecido no passado um irmao mais velho ardoroso e irrequie to, mas tambem cheio de brincadeiras animadas, uma enorme afinidade e uma infi nita capacidade de admiracao. 


## Remover Stopwords

In [7]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import spacy

# Carregar modelo de português
nlp = spacy.load("pt_core_news_sm")

# Baixar recursos necessários (roda apenas 1 vez)
nltk.download("punkt")
nltk.download("punkt_tab")
nltk.download("stopwords")
stopwords_pt = set(stopwords.words("portuguese"))


token = nlp(text_clean)
tokens_no_stop = [t for t in token if t.text not in stopwords_pt]

print("Tokens sem stopwords (primeiros 30):\n")
print(tokens_no_stop[:30])


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\GamingPc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\GamingPc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\GamingPc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Tokens sem stopwords (primeiros 30):

[prologo, coracao, fanatico, theo, imaginou, pior, ., mensagem, dizia, apenas, vincent, ferido, ., enquanto, corria, ate, estacao, pegar, proximo, trem, ate, auvers, ,, pensa, mentos, vagavam, tras, diante, ., ultima]


## Lemmatização

In [ ]:


def lemmatize_text(tokens):
    return [token.lemma_ for token in tokens]

lemmas = " ".join(lemmatize_text(tokens_no_stop))

print("Lemmas (primeiros 30):\n")
print(lemmas[:300])


Lemmas (primeiros 30):

prologo coracao fanatico theo imaginar mau . mensagem dizer apenas vincent ferir . enquanto correr ate estacao pegar proximo tr ate auver , pensar mento vagar tra diante . ultimo vez haver receber menso gem parecir , telegrama paul gauguin informar vincent gravemente enfermo . theo chegar cidade mer


## Stemming

In [189]:
# --- STEMMATIZAÇÃO (alternativa à lematização) ---
from nltk.stem.snowball import SnowballStemmer

# Criar stemmer para português
stemmer = SnowballStemmer("portuguese")

# Aplicar stemmatização nos tokens sem stopwords
stems = " ".join([stemmer.stem(token) for token in tokens_no_stop])

print("Stems (primeiros 30):\n")
print(stems[:300])


Stems (primeiros 30):

prolog coraca fanat the imagin pior . mensag diz apen vincent fer . enquant corr ate estaca peg proxim trem ate auvers , pens ment vag tras diant . ultim vez hav receb mens gem parec , telegram paul gauguin inform vincent gravement enferm . the cheg cidad meridional arles encontr irma enferm hospita


# Chunking


### Por Tamanho Fixo

In [119]:
# --- CHARACTER TEXT SPLITTER ---
from langchain_text_splitters import CharacterTextSplitter

char_splitter = CharacterTextSplitter(
    chunk_size=150,
    chunk_overlap=10,
    separator=".",
    length_function=len
)

chunks_char = char_splitter.split_text(lemmas)

print(f"Total de chunks: {len(chunks_char)}")
print(chunks_char)


Total de chunks: 7
['titulo introducao machine Learning machine learning Campo inteligencia artificial desenvolver algoritmos capaz aprender padroe partir dado', 'principal tipo incluir aprendizar supervisionar , aprendizar nao supervisionar aprendizar reforco', 'aprendizar supervisionar envolver treinar modelo dado rotular , prever preco casa base caracteristico tamanho , localizacaar numero quarto', 'aprendizar nao supervisionar detecta padroe oculto dar nao rotular , segmentacao cliente Marketing', 'aprendizar reforco ensina agente tomar decisoes ambiente dinamico maximizar recompensa', 'rede neural profundo sao usar visao computacional processamento linguagem natural', 'modelo cnns transformer revolucionar tarefa traducao automatica , reconhecimento imagem geracaar texto']


### Recursívo

In [36]:
# --- RECURSIVE CHARACTER TEXT SPLITTER ---
from langchain_text_splitters import RecursiveCharacterTextSplitter

recursive_splitter = RecursiveCharacterTextSplitter(
    chunk_size=150,
    chunk_overlap=10,
    separators=["\n\n", "\n", ".", ",", " ", ""]
)

chunks_recursive = recursive_splitter.split_text(lemmas)

print(f"Total de chunks: {len(chunks_recursive)}")
print(chunks_recursive)


Total de chunks: 222
['prologo coracao fanatico theo imaginar mau . mensagem dizer apenas vincent ferir', '. enquanto correr ate estacao pegar proximo tr ate auver , pensar mento vagar tra diante', '. ultimo vez haver receber menso gem parecir , telegrama paul gauguin informar vincent gravemente enfermo', '. theo chegar cidade meridional arle encontrara irmao enfermaria hospital , cabeca enfaixar perder desvario', '. dessa vez , encontrar final viagem momento assim muito , theo retornar lembranca vincent conhecer passado irmao velho ardoroso irrequie to', ', tambem cheio brincadeira animar , enorme afinidade infi nito capacidade admiracao', '. passeio infancia campo mata redor cidade holandês zundert , onde haver nascir , vincent apresentar beleza misterio natureza', '. inverno , vincent ensinar patinar andar treno . verao , mostrar construir castelo trilha areia', '. igreja domingo casa piano sala , cantar voz firme limper', '. quarto sotao dividiar , conversar ate tarde noite , criar

# Embedding

## Hugging Face SentenceTransformers

In [37]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("rufimelo/bert-base-portuguese-cased-sts")

def embed_hf(chunks):
    vectors = model.encode(chunks, show_progress_bar=True)

    print(f"Dimensão: {vectors.shape}")
    print(vectors[:5])

    return vectors

# Testes
embedding = embed_hf(chunks_recursive)


Invalid model-index. Not loading eval results into CardData.
Batches: 100%|██████████| 7/7 [00:11<00:00,  1.59s/it]

Dimensão: (222, 1024)
[[-0.21250078 -0.75260246 -0.43712464 ...  0.9434719   0.34019727
   0.20820548]
 [-1.0574208  -0.00806094 -0.15495598 ...  0.42858157 -0.5888575
   0.56559896]
 [ 0.34931388 -0.4595157  -0.27976772 ...  1.1793864  -0.572575
   0.18721654]
 [-0.9542238  -0.5475875   0.68637687 ... -0.45587382  0.08488183
  -0.7109255 ]
 [-0.8502135   0.05601178  0.7732106  ...  0.58355194 -0.6356497
   0.13664456]]


# Banco Vetorial

## Criação/Inserção ChromaDb

In [17]:
import chromadb

client = chromadb.Client()

#client.delete_collection("exemplo_colecao")

collection = client.get_or_create_collection(
    name="exemplo_colecao",
    embedding_function=None,
    # metadata={"hnsw:space": "cosine"}   # métrica mais comum
)
print(f"{collection.count()} documentos na coleção")

222 documentos na coleção


In [16]:
ids = [f"chunk_{i}" for i in range(len(chunks_recursive))]

collection.add(
    ids=ids,
    documents=chunks_recursive,
    embeddings=embedding
)

print("Embeddings inseridos com sucesso!")


Embeddings inseridos com sucesso!


## Busca Semântica

In [18]:
question = "Van Gogh?"
question_embedding = model.encode([question])

resultados = collection.query(
    query_embeddings=question_embedding.tolist(),
    n_results=3
)

print(f"\nPergunta: {question}")

for doc, score in zip(resultados['documents'][0], resultados['distances'][0]):
    print(f"(Score: {score}) - Documento: {doc}")


Pergunta: Van Gogh?
(Score: 435.5587158203125) - Documento: . ninguem acreditar importancia biografia fervor grande vincent van gogh
(Score: 450.1562194824219) - Documento: . Willem , cinco ano idade , avo pintor vincent Willem van gogh
(Score: 538.0499877929688) - Documento: , missao evangelico andarilho , incursao ilustracoes revista , finalmente breve fulgurante carreirar pintor
